In [1]:
import sys
import os

# Add project root to sys.path
sys.path.append(os.path.abspath(os.path.join("../..")))  # if you're in optimizer/

import pandas as pd
from source.utils import sample_from_distribution
import random
import numpy as np
import datetime
import pytz
import itertools
from collections import defaultdict


Load stuff

In [2]:
import pickle

with open("../parameters/simulation_parameters.pkl", "rb") as f:
    sim_params = pickle.load(f)

Explore prefixes

In [136]:
transition_probabilities = sim_params['transition_probabilities']
agent_transition_probabilities = sim_params['agent_transition_probabilities']

In [125]:
def compute_transition_weight(
    performed: list[str],
    last_agent: int,
    transition_probabilities: dict,
    agent_transition_probabilities: dict,
    is_orchestrated: float,
    proposal_agent: int,
    activity: str,
) -> float:
    """
    Weights regarding historic behavior:
    - If the process is autonomous then we only need the agent_transition_probabilities
    - If the process is orchestrated then we need to use transition_probabilities

    Weights regarding optimization:
    - tbd

    Returns:
    - joint_waigt: float
    """
    if not performed:
        return 0.0  # First activity → no context for prediction
    
    if is_orchestrated:
        prefix = tuple(performed)
        # P(activity | prefix, agent)
        agent_sim_weight = transition_probabilities.get(prefix, {}).get(proposal_agent, {}).get(activity, 0.0)
    else:
        last_activity = performed[-1]
        # P(agent, activity | last_agent, last_activity)
        agent_sim_weight = agent_transition_probabilities.get(last_agent, {}).get(last_activity, {}).get(proposal_agent, {}).get(activity, 0.0)

    transition_weight = agent_sim_weight
    
    return transition_weight

In [ ]:
performed_activities=  ['Check application form completeness']
last_agent = 10
transition_probabilities = sim_params['transition_probabilities']
agent_transition_probabilities = sim_params['agent_transition_probabilities']
proposal_agent = 1
proposed_activity = 'Check credit history'
is_orchestrated = sim_params['central_orchestration']


In [132]:
compute_transition_weight(
    performed_activities,
    last_agent,
    transition_probabilities,
    agent_transition_probabilities,
    is_orchestrated,
    proposal_agent,
    proposed_activity,
) 
    

0.025

In [143]:
import random

p = ['1','2','3']
w = [0.034, 0.009, 0.06]


for i in range(0,10):
    res = random.choices(p, weights=w, k=1)
    print(res)



['1']
['3']
['1']
['3']
['1']
['3']
['3']
['3']
['1']
['2']


In [152]:
agent_transition_probabilities[12]['Check application form completeness']

{2: {'AML check': 0.14166666666666666},
 3: {'Appraise property': 0.20833333333333334},
 7: {'AML check': 0.125},
 10: {'Check credit history': 0.025},
 6: {'Appraise property': 0.13333333333333333},
 5: {'Check credit history': 0.03333333333333333,
  'Return application back to applicant': 0.008333333333333333},
 8: {'Return application back to applicant': 0.041666666666666664,
  'Check credit history': 0.09166666666666666},
 9: {'Return application back to applicant': 0.025,
  'Check credit history': 0.025},
 4: {'Check credit history': 0.041666666666666664,
  'Return application back to applicant': 0.016666666666666666},
 1: {'Check credit history': 0.041666666666666664,
  'Return application back to applicant': 0.008333333333333333},
 0: {'Return application back to applicant': 0.008333333333333333,
  'Check credit history': 0.025}}

In [153]:
agent_transition_probabilities[9]['Check application form completeness']

{7: {'AML check': 0.14754098360655737},
 1: {'Check credit history': 0.06557377049180328,
  'Return application back to applicant': 0.01639344262295082},
 2: {'AML check': 0.16393442622950818},
 10: {'Check credit history': 0.05737704918032787,
  'Return application back to applicant': 0.04918032786885246},
 6: {'Appraise property': 0.09016393442622951},
 8: {'Check credit history': 0.04918032786885246,
  'Return application back to applicant': 0.00819672131147541},
 4: {'Return application back to applicant': 0.00819672131147541,
  'Check credit history': 0.06557377049180328},
 3: {'Appraise property': 0.14754098360655737},
 12: {'Return application back to applicant': 0.00819672131147541,
  'Check credit history': 0.04918032786885246},
 5: {'Check credit history': 0.03278688524590164,
  'Return application back to applicant': 0.01639344262295082},
 0: {'Check credit history': 0.02459016393442623}}

In [148]:
sim_params['agent_transition_probabilities'][4]['Check application form completeness']

{6: {'Appraise property': 0.11818181818181818},
 10: {'Check credit history': 0.03636363636363636,
  'Return application back to applicant': 0.01818181818181818},
 5: {'Check credit history': 0.06363636363636363,
  'Return application back to applicant': 0.00909090909090909},
 7: {'AML check': 0.19090909090909092},
 8: {'Check credit history': 0.07272727272727272,
  'Return application back to applicant': 0.00909090909090909},
 2: {'AML check': 0.1},
 3: {'Appraise property': 0.13636363636363635},
 0: {'Check credit history': 0.07272727272727272,
  'Return application back to applicant': 0.00909090909090909},
 12: {'Check credit history': 0.03636363636363636,
  'Return application back to applicant': 0.03636363636363636},
 9: {'Check credit history': 0.02727272727272727,
  'Return application back to applicant': 0.00909090909090909},
 1: {'Return application back to applicant': 0.00909090909090909,
  'Check credit history': 0.045454545454545456}}

In [9]:
probabilities = sim_params['agent_transition_probabilities']
probabilities[0].keys()

dict_keys(['Check application form completeness', 'Cancel application', 'Check credit history', 'Reject application', 'Approve application', 'Return application back to applicant'])

In [11]:
probabilities[0]['Check application form completeness']

{1: {'Check credit history': 0.07377049180327869,
  'Return application back to applicant': 0.00819672131147541},
 6: {'Appraise property': 0.1557377049180328},
 3: {'Appraise property': 0.16393442622950818},
 5: {'Check credit history': 0.05737704918032787},
 4: {'Check credit history': 0.05737704918032787,
  'Return application back to applicant': 0.00819672131147541},
 2: {'AML check': 0.13934426229508196},
 8: {'Check credit history': 0.08196721311475409},
 12: {'Return application back to applicant': 0.01639344262295082,
  'Check credit history': 0.02459016393442623},
 9: {'Return application back to applicant': 0.03278688524590164,
  'Check credit history': 0.04918032786885246},
 10: {'Check credit history': 0.02459016393442623,
  'Return application back to applicant': 0.00819672131147541},
 7: {'AML check': 0.09836065573770492}}

In [ ]:
sim_params['max_activity_count_per_case'][]

{'AML check': 1,
 'Applicant completes form': 3,
 'Appraise property': 1,
 'Approve application': 1,
 'Approve loan offer': 1,
 'Assess loan risk': 1,
 'Cancel application': 1,
 'Check application form completeness': 4,
 'Check credit history': 1,
 'Design loan offer': 1,
 'Reject application': 1,
 'Return application back to applicant': 3,
 'zzz_end': 1}